<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Raleway:wght@100&display=swap" rel="stylesheet">

<h1 style="text-transform: uppercase; text-align: center; font-weight: 100;">
Linear regression in Tensorflow using RNN</h1>
<p style="text-align: center; font-weight: 100;">Predicting Volkswagen car prices</p><br/>

Project requirements:
- Only requirement is: minimum 10k rows
- Any type of neural network - classical but convolutional and recurrent, better grade
- Presentation of project, beside practical questions about model there would be also theoretical.
The topic is free.

Theory required for project presentation:
- Practical side.
- Discuss the scheme of an artificial neuron.
- Activation function, why it is so important.
- Explain how does neuron learn - steps, algorithm?
- How does basic neural network work? (The more you know the better, as well as more complex).
- Discuss the algorithm of neural network using backpropagation learning method. (metoda wstecznej propagacji błędów)
- What subsets and why are the data divided into?
Knowledge of ML, statistics.

Data source: data were taken from the popular Polish automotive website Otomoto (https://www.otomoto.pl/).

1. Import libraries and create dataframe:

In [ ]:
import sklearn as sl
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
import tensorflow as tf
import keras.utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# from model import VolkswagenModel
# from colorama import init, Fore, Style

print(f"numpy: {np.__version__}, pandas: {pd.__version__}, tensorflow: {tf.__version__}, matplotlib: {mpl.__version__}, seaborn: {sns.__version__}, statsmodels: {sm.__version__}, sklearn: {sl.__version__}")
print("Libs loaded.")

In [ ]:
data = pd.read_csv("./data/otomoto.csv")
df = pd.DataFrame(data)
df = df.drop(df.columns[0], axis=1)
df.columns = ['Price', 'Year', 'Mileage', 'Tank capacity', 'Fuel type', 'Model', 'Estimation']
print("Data technical info")
df.info()
df

In [ ]:
description = df.describe()
description

In [ ]:
check = 14915
flag = True
for i in range(description.shape[1]):
    if description.iloc[0, i] != check:
        print("Number of occurrences of data is not equal for every label.")
        print(f"Problem at cell: (0, {i})")
        flag = False
        continue
    else:
        print(f"Checked: (0, {i})")

print("All columns passed" if flag == True else "Not passed")

2. Clean data:

    Data was cleaned previously in 'scratchpad.py' file and now all the records are represented by integers(documentation of each column values is located in 'model.py' file). Values ​​of 0 represent an error in reading data. Column "Estimation" contains a lot of 0 values, but this is due to the fact that not every article on the website contained such information.

In [ ]:
if df.isna().any().any():
    df = df.dropna()
    print("All rows with NaN values were dropped")
else:
    print("0 NaN values")

cols_to_check = df.columns[df.columns != 'Estimation']
if df[cols_to_check].eq(0).any().any():
    df[cols_to_check] = df[cols_to_check].replace(0, np.nan)
    print("All rows with 0 values were dropped")
else:
    print("0 zero values")

In [ ]:
df


Getting know data, analyzing dependencies


Histograms

In [ ]:
def make_hist(col_name, bins_val):
    min = df[col_name].min()
    max = df[col_name].max()
    print(f"Lowest {col_name} value: {min}. Highest {col_name} value: {max}.")
    plot_hist = df[col_name].plot.hist(bins=bins_val, grid=True)
    plot_hist.set_title(f"Represents number of cars for each production {col_name.upper()} category")
    plot_hist.set_xlabel(f"{col_name}")
    plot_hist.set_ylabel("Number of observations")
    plt.show()

In [ ]:
make_hist("Price", 57)

In [ ]:
min = df["Year"].min()
max = df["Year"].max()
make_hist("Year", max-min)

In [ ]:
make_hist("Mileage", 70)

In [ ]:
make_hist("Tank capacity", 10) # default bins = 10

# Trzeba jeszcze dopasować przedziały

In [ ]:
make_hist("Fuel type", 10) # default bins = 10

In [ ]:
NO_cars_fuel = df['Fuel type'].value_counts().reset_index()
NO_cars_fuel = NO_cars_fuel.rename(columns={'index': 'type of fuel', 'Fuel type': 'NO of cars with specific fuel type'})
# NaN
# d = {"Benzyna": 1, "Benzyna+LPG": 2, "Benzyna+CNG": 3, "Elektryczny": 4, "Hybryda": 5, "Wodór": 6, "Diesel": 7}
# unique_values['name of fuel'] = unique_values['type of fuel'].map(d)
NO_cars_fuel

In [ ]:
make_hist("Model", 29)

In [ ]:
make_hist("Estimation", 10) # default bins = 10

Distribution of cars per column per certain category from column in numbers.

In [ ]:
# pr = df['Price'].value_counts().reset_index()
# yr = df['Year'].value_counts().reset_index()
# mil = df['Mileage'].value_counts().reset_index()
# tank = df['Tank capacity'].value_counts().reset_index()
# fuel = df['Fuel type'].value_counts().reset_index()
mod = df['Model'].value_counts().reset_index()
est = df['Estimation'].value_counts().reset_index()

# pr = pr.rename(columns={'index': 'price_cat', 'Price': 'NO_cars'})
# yr = yr.rename(columns={'index': 'year_cat', 'Year': 'NO_cars'})
# mil = mil.rename(columns={'index': 'mileage_cat', 'Mileage': 'NO_cars'})
# tank = tank.rename(columns={'index': 'tank_capacity', 'Tank capacity': 'NO_cars'})
# fuel = fuel.rename(columns={'index': 'type_fuel', 'Fuel type': 'NO_cars'})
mod = mod.rename(columns={'index': 'type_model', 'Model': 'NO_cars'})
est = est.rename(columns={'index': 'type_est', 'Estimation': 'NO_cars'})

def create_table(table_data, table_title):
    fig = plt.figure()
    table = plt.table(cellText=table_data.values, colLabels=table_data.columns, loc='upper left')
    table.auto_set_font_size(False)
    table.set_fontsize(14)
    table.scale(1, 1.5)
    plt.subplots_adjust(left=0.2, bottom=0.2)
    plt.title(table_title)
    plt.axis('off')
    return fig

# table1 = create_table(pr, "Pr")
# table2 = create_table(yr, "Yr")
# table3 = create_table(mil, "Mil")
# table4 = create_table(tank, "Tank")
# table5 = create_table(fuel, "Fuel")
table6 = create_table(mod, "Mod")
table7 = create_table(est, "Est")

Box plots

In [ ]:
for i, column in enumerate(df.columns):
    fig, ax = plt.subplots()
    sns.boxplot(data=df[column], ax=ax).set(title=f"{column.upper()} boxplot", xlabel=f"{column}", ylabel=f"Value of {column}")
    ax.set_title(column)
    sns.despine()
    plt.show()

Heat Map

In [ ]:
correlation_matrix = np.corrcoef(df.values.T)

fig, ax = plt.subplots(figsize=(7, 7))
sns.set(font_scale=1.1)
sns.heatmap(data=correlation_matrix, square=True, cbar=True, annot=True, annot_kws={'size': 10}, xticklabels=df.columns, yticklabels=df.columns, fmt=".2f", linewidth=.5, cmap=sns.cubehelix_palette(as_cmap=True))

Outlier detection

In [ ]:
# https://www.kaggle.com/code/durgeshrao9993/removing-outliers-by-iqr-method

# # Interquartile Range (IQR)
# Q1 = data.quantile(0.25)
# Q3 = data.quantile(0.75)
# IQR = Q3 - Q1
# data = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]

# Calculate the z-scores of each column - measure that represents the number of standard deviations a data point is from the mean of the dataset
z_scores = (df - df.mean()) / df.std()
# Set the threshold for the z-score
threshold = 3
# Remove any rows where the z-score is greater than the threshold
df = df[(np.abs(z_scores) < threshold).all(axis=1)]
df

In [ ]:
print("14915 dropped to 14105 rows")

Heat Map - after outlier detection

In [ ]:
sns.boxplot(df['Price']).set(title="PRICE boxplot", xlabel="Price", ylabel="Value of Price")
sns.despine()

In [ ]:
sns.boxplot(df['Year']).set(title="YEAR boxplot", xlabel="Year", ylabel="Value of Year")
sns.despine()

In [ ]:
sns.boxplot(df['Mileage']).set(title="MILEAGE boxplot", xlabel="Mileage", ylabel="Value of Mileage")
sns.despine()

In [ ]:
sns.boxplot(df['Tank capacity']).set(title="TANK CAPACITY boxplot", xlabel="Tank capacity", ylabel="Value of Tank capacity")
sns.despine()

Splitting data into sets

In [ ]:
# SKLEARN

X = df.drop(['Price'], axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# KERAS/TENSORFLOW

# data_size = len(df)
# test_size = int(data_size * 0.2)  # 20% danych przeznaczamy na zbiór testowy
# train_size = data_size - test_size
#
# data = tf.data.Dataset.from_tensor_slices((X.values, y.values))
# data = data.shuffle(buffer_size=data_size, reshuffle_each_iteration=True) # randomize data after each iteration (epoch)
#
# # Helps to optimize code by dividing datasets into batches, portion of data instead of individual processing.
# train_data = data.take(train_size).batch(32)
# test_data = data.skip(train_size).take(test_size).batch(32)
# X_train, y_train = next(iter(train_data))
# X_test, y_test = next(iter(test_data))
#
# # Print length of datasets portions
# print(f"X_train len = {len(X_train)}")
# print(f"y_train len = {len(y_train)}")
# print(f"X_test len = {len(X_test)}")
# print(f"y_test len = {len(y_test)}")
#
# # Print datasets portions
# # print('X_train = ', X_train)
# # print('y_train = ', y_train)
# # print('X_test = ', X_test)
# # print('y_test = ', y_test)

Normalization / Scaling

In [ ]:
# Normalization - we change the distribution of data, not the range of data like it is done in scaling. The point is to change your observations so that they can be described as a normal distribution.
# Normalization should be done after splitting into train and test data sets!!!
# https://keras.io/api/layers/preprocessing_layers/numerical/normalization/
# fit_transform() is used to learn and apply the transformation to the data in one step, whereas transform() is used to apply the transformation to new data using the learned parameters.
# If you are using a regression model and want to predict a continuous target variable (i.e., y), then you should scale both the input features (X) and the target variable (y). This is because scaling both the input features and target variable will help the model to learn better and converge faster.
# If you are using a classification model and want to predict a binary or categorical target variable, then you do not need to scale the target variable (y). This is because scaling does not change the nature of the target variable, and most classification algorithms do not depend on the scale of the target variable.

scaler = MinMaxScaler()
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train_scaled = scaler.transform(y_train)
y_test_scaled = scaler.transform(y_test)
# convert dataframe to numpy array
# values = df.values
# normalized_values = keras.utils.normalize(values, axis=0)
# df_normalized = pd.DataFrame(normalized_values, columns=df.columns)
# print("Normalization done")

Pair plot

In [ ]:
sns.pairplot(df, height=1.5)

Model Architecture

In [ ]:
# https://www.tensorflow.org/guide/keras/rnn?hl=pl
# https://www.ibm.com/topics/recurrent-neural-networks

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(None, X_train.shape[-1])), # type of recurrent neural network
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Equivalent to:
# input_layer = tf.keras.layers.Input(shape=(None, X_train.shape[-1]))
# lstm_layer = tf.keras.layers.LSTM(64)(input_layer)
# dense_layer_1 = tf.keras.layers.Dense(32, activation='relu')(lstm_layer)
# output_layer = tf.keras.layers.Dense(1)(dense_layer_1)
# model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

Compile and train training the model

In [ ]:
model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=5,
    validation_data=(X_test_scaled, y_test_scaled),
    batch_size=32,
    validation_split=0.2
)

Evaluating the model

In [ ]:
# Evaluate the model on the training set
train_loss, train_mse, train_mae, train_r2, train_acc = model.evaluate(X_train_scaled, y_train_scaled, verbose=0)
print("Training Loss: {:.4f}".format(train_loss))
print("Training MSE: {:.4f}".format(train_mse))
print("Training MAE: {:.4f}".format(train_mae))
print("Training R2: {:.4f}".format(train_r2))
print("Training Accuracy: {:.4f}".format(train_acc))

# Evaluate the model on the validation (test) set
# "Generally, the term “validation set” is used interchangeably with the term “test set”"
val_loss, val_mse, val_mae, val_r2, val_acc = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print("Validation Loss: {:.4f}".format(val_loss))
print("Validation MSE: {:.4f}".format(val_mse))
print("Validation MAE: {:.4f}".format(val_mae))
print("Validation R2: {:.4f}".format(val_r2))
print("Validation Accuracy: {:.4f}".format(val_acc))

Loss and accuracy plots

In [ ]:
# Plot the loss function
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss function')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot the accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()


# Plot the loss and accuracy curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

Get the best hyperparameters

### Possible DEPLOY - docker, github